# Parents to Child mappings in StyleGAN2 latent space using the Ridge Regression

In [1]:
from dataset.nok_mean import NokMeanDataset
from dataset.nok import NokDataset
from dataset.nok_aug import NokAugDataset
from utils.stylegan import StyleGAN2
from utils.eval import BaseEvaluator
from utils.viz import image_add_label
import utils.nokdb as nokdb
import utils.ppldb as ppldb
import os
import shutil
import sys
import numpy as np
import pandas as pd
import torch, torchvision
from PIL import Image
from glob import glob
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
root = os.getenv('ROOT')


/home/vidp/miniconda3/envs/diploma_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/vidp/miniconda3/envs/diploma_env/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/vidp/miniconda3/envs/diploma_env/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
name = "parents_to_child"
output_path = "./src/.tmp/stylegan2-ada-pytorch/" + name + "001"
while (os.path.exists(output_path)):
  output_path = output_path[:-3] + str(int(output_path[-3:]) + 1).zfill(3)

os.mkdir(output_path)
output_path

'./src/.tmp/stylegan2-ada-pytorch/parents_to_child019'

In [3]:
# python src/utils/align_faces_parallel.py --root_path ./src/dataset/ppldb/vid/
for person in glob(f"./src/dataset/ppldb/people/*"):
    person_name = person.split("/")[-1]
    if not os.path.exists(f"{person}/aligned"):
        for i, image in enumerate(glob(f"{person}/*")):
            os.rename(image, f"{person}/{person_name}_{i}.jpg")
        os.system(f"python src/utils/align_faces_parallel.py --root_path {person}")

In [4]:
stylegan = StyleGAN2(tmp_path=output_path)
people = []
for person in glob(f"./src/dataset/ppldb/people/*"):
  if os.path.isdir(person) and os.path.exists(f"{person}/aligned") and not os.path.exists(f"{person}/latents"):
    people.append(stylegan.project_person(person))

print(people)
# /home/vidp/Documents/fri-2022-diploma/submodules/stylegan2-ada-pytorch/projector.py

./src/submodules/stylegan2-ada-pytorch
./src/.tmp/stylegan2-ada-pytorch/parents_to_child019
Got StyleGAN2 docker client, building image...
StyleGAN2 Docker image built.
[INFO] StyleGAN2 - Projecting image...
['python3', './src/submodules/stylegan2-ada-pytorch/projector.py', '--network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl', '--target=./src/dataset/ppldb/people/bradley/aligned/bradley_1.jpg', '--outdir=./src/dataset/ppldb/people/bradley/latents/latents_bradley_1', '--save-video=False']
Loading networks from "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"...

Computing W midpoint and stddev using 10000 samples...

Setting up PyTorch plugin "bias_act_plugin"... Done.

Setting up PyTorch plugin "upfirdn2d_plugin"... Done.

step    1/1000: dist 0.62 loss 24567.30

step    2/1000: dist 0.75 loss 27640.75

step    3/1000: dist 0.69 loss 27166.61

step    4/1000: dist 0.58 loss 26253.11

step    5/1000: dist 0.61 loss 24959.01



In [5]:
nokdb_sample_filename = "nokdb-samples-real-c2p"
for person in glob(f"./src/dataset/ppldb/people/*"):
    person_name = person.split("/")[-1]
    people_names = nokdb.people_names()
    if person_name not in people_names and os.path.exists(f"{person}/latents"):
        pid = nokdb.max_nokdb_pid() + 1
        pid
        ppldb.add_ppldb_nokdb_mapping(person_name, pid)
        os.mkdir(f"./src/dataset/nokdb/{pid}")
        # person = pid,name,family_name,sex,father_pid,mother_pid,race,sex_code,race_code
        person_row = [pid, person.split("/")[-1], "", "", "", "", "", "", ""]
        nokdb.add_person(person_row)
        for latent_folder in glob(f"{person}/latents/*"):
            if not os.path.isdir(latent_folder):
                continue
            target_img = f"{latent_folder}/target.png"
            latent = f"{latent_folder}/projected_w.npz"
            iid = nokdb.max_nokdb_iid() + 1
            shutil.copy(target_img, f"./src/dataset/nokdb/{pid}/{iid}.png")
            shutil.copy(latent, f"./src/dataset/nokdb/{pid}/{iid}.npz")
            # image = iid,pid,age,emotion,emotion_code
            image_row = [iid, pid, "", "", ""]
            nokdb.add_image(image_row)
            sample_row = ["", "", "", "", pid, iid]
            nokdb.add_sample(nokdb_sample_filename, sample_row)


In [6]:
couples = ppldb.couple_to_nokdb(2)
couples


1127 1128 [6992, 6993, 6994] [6995, 6996]
1127 6992 1128 6995  
1127 6992 1128 6996  
1127 6993 1128 6995  
1127 6993 1128 6996  
1127 6994 1128 6995  
1127 6994 1128 6996  


[[1127, 6992, 1128, 6995, '', ''],
 [1127, 6992, 1128, 6996, '', ''],
 [1127, 6993, 1128, 6995, '', ''],
 [1127, 6993, 1128, 6996, '', ''],
 [1127, 6994, 1128, 6995, '', ''],
 [1127, 6994, 1128, 6996, '', '']]

In [16]:
def load_data(split):
    X = []
    y = []
    for father, mother, child, child_gender, f_pid, m_pid, c_pid in NokMeanDataset(split=split):
        input = torch.cat([father.flatten(0), mother.flatten(0)], dim=0)
        output = child.flatten(0)
        X.append(input)
        y.append(output)
    X = torch.stack(X, dim=0)
    y = torch.stack(y, dim=0)
    return X, y

In [17]:
def load_data_real(split):
    X = []
    for father, mother, child, child_gender, f_pid, m_pid, c_pid in NokMeanDataset(split=split):
        input = torch.cat([father.flatten(0), mother.flatten(0)], dim=0)
        X.append(input)
    X = torch.stack(X, dim=0)
    return X

In [18]:
X_train, y_train = load_data(split="train")
X_test, y_test = load_data(split="test")
X_real = load_data_real(split="real-p2c")
X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_real.shape

Loaded 433 persons with 2676 images.
Average images per person: 6.180138568129331
Max images per person: 37
Min images per person: 1
[13, 12, 6, 1, 4, 2, 6, 9, 8, 2, 4, 7, 1, 6, 3, 3, 5, 8, 12, 4, 3, 6, 20, 4, 10, 3, 7, 3, 1, 4, 5, 9, 5, 8, 2, 4, 4, 3, 3, 2, 11, 2, 7, 14, 8, 10, 2, 1, 2, 9, 11, 5, 1, 1, 5, 2, 5, 5, 8, 1, 1, 5, 5, 3, 1, 3, 7, 23, 14, 15, 16, 5, 4, 1, 3, 9, 5, 6, 5, 1, 2, 5, 16, 4, 2, 1, 5, 2, 6, 24, 5, 15, 3, 9, 13, 2, 5, 6, 9, 7, 4, 8, 4, 7, 5, 11, 4, 9, 7, 6, 6, 7, 8, 5, 7, 5, 18, 10, 6, 8, 1, 5, 5, 9, 14, 6, 6, 1, 11, 14, 2, 7, 1, 2, 4, 4, 17, 5, 1, 6, 6, 4, 2, 3, 4, 5, 1, 2, 4, 4, 9, 3, 6, 2, 3, 5, 8, 6, 7, 14, 6, 5, 18, 9, 5, 3, 13, 3, 12, 5, 14, 5, 5, 1, 5, 9, 2, 9, 2, 6, 10, 8, 10, 12, 9, 6, 19, 10, 7, 37, 9, 6, 1, 1, 4, 24, 7, 3, 3, 4, 6, 13, 5, 8, 4, 5, 4, 2, 24, 5, 19, 6, 5, 13, 5, 3, 7, 8, 11, 4, 12, 7, 8, 7, 14, 15, 6, 8, 4, 5, 3, 5, 6, 8, 11, 6, 2, 2, 10, 4, 4, 3, 1, 1, 1, 1, 2, 1, 2, 5, 1, 16, 9, 8, 4, 4, 1, 9, 3, 5, 7, 8, 3, 2, 6, 4, 1, 5, 2, 4, 5, 6, 4, 

(torch.Size([207, 18432]),
 torch.Size([207, 9216]),
 torch.Size([20, 18432]),
 torch.Size([20, 9216]),
 torch.Size([1, 18432]))

In [19]:
regressor = Ridge()
regressor.fit(X_train, y_train)

Ridge()

In [20]:
y_train_hat = regressor.predict(X_train)
y_test_hat = regressor.predict(X_test)
y_real_hat = regressor.predict(X_real)
y_test_hat, y_real_hat

(array([[-0.03976554, -0.25728131,  1.02684866, ...,  0.1580961 ,
         -0.27821751, -0.08716307],
        [ 0.21214497,  0.56130235,  0.69493189, ..., -0.03583013,
         -0.19473159, -0.18125735],
        [ 0.05996377,  0.57765832,  0.63638776, ...,  0.15714133,
         -0.16814867, -0.23706346],
        ...,
        [ 0.31836431,  0.01427651,  1.18794338, ..., -0.58418127,
          0.2748577 , -0.12946716],
        [ 0.72618746,  0.37628599,  0.95541027, ..., -0.6393806 ,
         -0.49785653, -0.04573409],
        [ 0.12426073, -0.11383929,  0.63160539, ..., -0.05098567,
         -0.44454404,  0.09901073]]),
 array([[-0.14495235,  1.02382685,  0.02859384, ..., -0.48262321,
         -0.12193367,  0.02391783]]))

In [21]:
mse_train = mean_squared_error(y_train, y_train_hat)
mse_test = mean_squared_error(y_test, y_test_hat)
mse_train, mse_test

(0.39116632147053854, 0.5975514193851812)

In [22]:
images = []
for i in range(y_real_hat.shape[0]):
    images.append(X_real[i,:18*512].view(18, 512))
    images.append(X_real[i,18*512:].view(18, 512))
    images.append(torch.from_numpy(y_real_hat[i]).to(torch.float32).view(18, 512))

images = torch.stack(images, dim=0)

In [23]:
images = []
for i in range(y_test.shape[0]):
    images.append(X_test[i,:18*512].view(18, 512))
    images.append(X_test[i,18*512:].view(18, 512))
    images.append(y_test[i].view(18, 512))
    images.append(torch.from_numpy(y_test_hat[i]).to(torch.float32).view(18, 512))

images = torch.stack(images, dim=0)

## Visualize and evaluate results

In [23]:
toTensor = torchvision.transforms.PILToTensor()
toPIL = torchvision.transforms.ToPILImage()
eval = BaseEvaluator()
stylegan = StyleGAN2(tmp_path=output_path)

./src/submodules/stylegan2-ada-pytorch
./src/.tmp/stylegan2-ada-pytorch/parents_to_child019
Got StyleGAN2 docker client, building image...
StyleGAN2 Docker image built.


In [24]:
rows = 3
pils = stylegan.generate_from_array(images.detach().cpu().numpy())
pil = toPIL(torchvision.utils.make_grid([toTensor(pil.resize((128,128))) for pil in pils], nrow=rows)).convert("RGB")
pil.save(output_path + "/result.png")

Generating 3 images from array...
[INFO] StyleGAN2 - Generating image...
['python3', './src/submodules/stylegan2-ada-pytorch/generate.py', '--network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl', '--outdir=./src/.tmp/stylegan2-ada-pytorch/parents_to_child019', '--noise-mode=random', '--projected-w=./src/.tmp/stylegan2-ada-pytorch/parents_to_child019/projected_w.npz']
Loading networks from "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"...

Generating images from projected W "./src/.tmp/stylegan2-ada-pytorch/parents_to_child019/projected_w.npz"

Setting up PyTorch plugin "bias_act_plugin"... Done.

Setting up PyTorch plugin "upfirdn2d_plugin"... Done.



In [36]:
pils_reshaped = np.array([np.array(pil).reshape(1024, 1024, 3) for pil in pils])
pils_reshaped = pils_reshaped.reshape(-1, 4, 1024, 1024, 3)
images_eval = pils_reshaped[:, 2]
images_hat_eval = pils_reshaped[:, 3]

In [37]:
images_eval_arr = [Image.fromarray(i).convert("RGB") for i in images_eval]
images_hat_eval_arr = [Image.fromarray(i).convert("RGB") for i in images_hat_eval]

In [38]:
images_eval_pil = toPIL(torchvision.utils.make_grid([toTensor(pil.resize((128, 128))) for pil in images_eval_arr], nrow=1)).convert("RGB")
images_eval_pil.save(output_path + "/images_eval.png")

In [45]:
eval_res_fn = eval.evaluate_batch(images_eval, images_hat_eval, model_name='Facenet512')
images_hat_eval_arr_labeled_fn = zip(images_hat_eval_arr, eval_res_fn)
images_hat_eval_pil_fn = toPIL(torchvision.utils.make_grid([toTensor(image_add_label(pil, str(round(label, 3)), 40).resize((256, 256))) for pil, label in images_hat_eval_arr_labeled_fn], nrow=1)).convert("RGB")
images_hat_eval_pil_fn.save(output_path + "/images_eval_hat_fn.png")
eval_res_fn

1/1 [==============================] - 0s 41ms/step


[0.33402624846914436,
 0.43684275283856516,
 0.159515057262758,
 0.22805098862428852,
 0.47382434400613477,
 0.2976961162220601,
 0.4468262297577832,
 0.4712513135122438,
 0.03458128400108661,
 0.3743983516671815,
 0.3978984547547002,
 0.16578259659631245,
 0.44721421017624513,
 -0.017200315587250783,
 0.5137009203637347,
 0.22086447394089861,
 -0.11366718482966436,
 0.0744433842380785,
 0.20400588503612396,
 -0.1373116028271647]

In [44]:
eval_res_af = eval.evaluate_batch(images_eval, images_hat_eval, model_name='ArcFace')
images_hat_eval_arr_labeled_af = zip(images_hat_eval_arr, eval_res_af)
images_hat_eval_pil_af = toPIL(torchvision.utils.make_grid([toTensor(image_add_label(pil, str(round(label, 3)), 40).resize((256, 256))) for pil, label in images_hat_eval_arr_labeled_af], nrow=1)).convert("RGB")
images_hat_eval_pil_af.save(output_path + "/images_eval_hat_af.png")
eval_res_af

1/1 [==============================] - 0s 67ms/step


[0.10377961691985067,
 0.943256346206428,
 -0.008764100807423717,
 0.21682055050580815,
 0.26986823526496284,
 0.1969762862823228,
 0.019916973462405136,
 0.358388616413088,
 0.0038190441572530888,
 0.3309051924515633,
 0.1511933387718012,
 0.03959055423529361,
 0.5192292305804449,
 0.025049313347839344,
 0.5997104295188008,
 0.10366729960530924,
 0.03381036896741243,
 0.05439476054490021,
 0.023167767168489554,
 0.07833462406799792]

In [43]:
eval_res_vgg = eval.evaluate_batch(images_eval, images_hat_eval, model_name='VGG-Face')
images_hat_eval_arr_labeled_vgg = zip(images_hat_eval_arr, eval_res_vgg)
images_hat_eval_pil_vgg = toPIL(torchvision.utils.make_grid([toTensor(image_add_label(pil, str(round(label, 3)), 40).resize((256, 256))) for pil, label in images_hat_eval_arr_labeled_vgg], nrow=1)).convert("RGB")
images_hat_eval_pil_vgg.save(output_path + "/images_eval_hat_vgg.png")
eval_res_vgg

1/1 [==============================] - 0s 174ms/step


[0.464754694541439,
 0.8274239937676159,
 0.34938368077529997,
 0.576459335828597,
 0.5537390224322192,
 0.2699485708488388,
 0.46675176195489104,
 0.6830751306542533,
 0.046867572628578304,
 0.7086720534966884,
 0.4166164836708298,
 0.5513433182114134,
 0.6468025871821815,
 0.2652512330690248,
 0.6530968067392857,
 0.6092680018630461,
 0.12331453282012529,
 0.25539691219079025,
 0.3061176826490259,
 0.10899034316594093]